In [15]:
import numpy
import os

import dicom2nifti # to convert DICOM files to the NIftI format
import nibabel as nib # nibabel to handle nifti files


In [16]:
import imageio

In [17]:
current_path = os.getcwd()
path_root = os.path.join(current_path, '../../CHAOS_Train_Sets/Train_Sets')
path_mr = os.path.join(path_root, "MR")
print(path_mr)
assert os.path.exists(path_mr), "Path does not exist"

/Users/wangzhengwei/Desktop/repo/SGMANet_new/dataset/../../CHAOS_Train_Sets/Train_Sets/MR


In [18]:
def convert_data(path_input, path_output, name=None):
    dicom2nifti.convert_directory(path_input, path_output)

    if name:
        for file in os.listdir(path_output):
            name_old = os.path.join(path_output, file)
            if name_old.split('.')[-2:] == ['nii','gz']:
                name_new = os.path.join(path_output, name + ".nii.gz")
                os.rename(name_old, name_new)

In [19]:
def convert_label(path_input, path_output, name, shape, affine):
    slice = os.listdir(path_input)
    slice = [s for s in slice if os.path.splitext(s)[-1]=='.png']
    print(slice)
    slice.sort()

    save_path = os.path.join(path_output, name + ".nii.gz")

    image_label = numpy.zeros(shape, dtype=numpy.uint8)

    for i,file in enumerate(slice):
        file_path = os.path.join(path_input, file)
        slice_data = imageio.v3.imread(file_path)
        image_label[:, :, i] = numpy.rot90(slice_data, k=1, axes=(1,0))

    label_numbers = numpy.unique(image_label)
    print(label_numbers)
    for new_number,label_number in enumerate(label_numbers):
        image_label[image_label == label_number] = new_number

    new_image = nib.Nifti1Image(image_label, affine)
    nib.save(new_image, save_path)

In [20]:
for folder in os.listdir(path_mr):
    path_mr_now = os.path.join(path_mr, folder)
    if os.path.isdir(path_mr_now):
        print(path_mr_now)
        path_mr_t1 = os.path.join(path_mr_now, "T1DUAL")
        path_mr_t1_label = os.path.join(path_mr_t1,'Ground')
        path_mr_t1_data = os.path.join(path_mr_t1, "DICOM_anon")
        path_ct_t1_data_inphase = os.path.join(path_mr_t1_data, "InPhase")

        path_mr_t2 = os.path.join(path_mr_now, "T2SPIR")
        path_mr_t2_label = os.path.join(path_mr_t2,'Ground')
        path_mr_t2_data = os.path.join(path_mr_t2, "DICOM_anon")

        convert_data(path_ct_t1_data_inphase, path_mr_t1_data, folder)
        image_data = nib.load(os.path.join(path_mr_t1_data, folder+".nii.gz"))
        convert_label(path_mr_t1_label, path_mr_t1_label, folder, image_data.shape, image_data.affine)

        convert_data(path_mr_t2_data, path_mr_t2_data, folder)
        image_data = nib.load(os.path.join(path_mr_t2_data, folder+".nii.gz"))
        convert_label(path_mr_t2_label, path_mr_t2_label, folder, image_data.shape, image_data.affine)       


/Users/wangzhengwei/Desktop/repo/SGMANet_new/dataset/../../CHAOS_Train_Sets/Train_Sets/MR/32
['IMG-0031-00018.png', 'IMG-0031-00024.png', 'IMG-0031-00030.png', 'IMG-0031-00026.png', 'IMG-0031-00032.png', 'IMG-0031-00036.png', 'IMG-0031-00022.png', 'IMG-0031-00008.png', 'IMG-0031-00034.png', 'IMG-0031-00020.png', 'IMG-0031-00052.png', 'IMG-0031-00046.png', 'IMG-0031-00050.png', 'IMG-0031-00044.png', 'IMG-0031-00040.png', 'IMG-0031-00054.png', 'IMG-0031-00042.png', 'IMG-0031-00056.png', 'IMG-0031-00066.png', 'IMG-0031-00064.png', 'IMG-0031-00058.png', 'IMG-0031-00060.png', 'IMG-0031-00048.png', 'IMG-0031-00062.png', 'IMG-0031-00010.png', 'IMG-0031-00004.png', 'IMG-0031-00038.png', 'IMG-0031-00012.png', 'IMG-0031-00006.png', 'IMG-0031-00002.png', 'IMG-0031-00016.png', 'IMG-0031-00014.png', 'IMG-0031-00028.png']
[  0  63 126 189 252]
['IMG-0030-00019.png', 'IMG-0030-00031.png', 'IMG-0030-00025.png', 'IMG-0030-00024.png', 'IMG-0030-00030.png', 'IMG-0030-00018.png', 'IMG-0030-00026.png', 'IM